# Cars Price Prediction

University of Colorado Boulder – CSCA5622  
**Author**: Adrian Gomez  
**Contact**: adrian.gomez-1@colorado.edu

### Next Steps
- Link to the project repository
- Document the final feature-selection rationale
- Decide whether to add a table of contents
- Summarize results in a concluding section


In [1]:
from pathlib import Path
import shutil

import kagglehub
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_theme(context='notebook', style='ticks')

/Users/adrianplattner/git/csca5922/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## The Problem

Used-car prices vary across regions, seasons, and listing platforms, making it difficult to establish a fair asking price. Buyers risk overpaying, while sellers may leave money on the table. A predictive model offers an evidence-based reference point that helps both parties negotiate confidently and reduce time spent sifting through listings.


## Dataset

Used Cars dataset on Kaggle: https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data/data


### Data Acquisition


In [2]:
DATA_PATH = Path('../data/vehicles.csv')
DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

if not DATA_PATH.exists():
    print('Local vehicles.csv not found; downloading with kagglehub...')
    download_dir = Path(kagglehub.dataset_download('austinreese/craigslist-carstrucks-data'))
    source_csv = download_dir / 'vehicles.csv'
    if not source_csv.exists():
        raise FileNotFoundError(
            f"'vehicles.csv' not present in downloaded dataset directory {download_dir}."
        )
    shutil.copy(source_csv, DATA_PATH)
    print(f'Copied dataset to {DATA_PATH.resolve()}')

df = pd.read_csv(DATA_PATH)

print(f'Loaded {len(df):,} rows and {df.shape[1]} columns from {DATA_PATH}')

Loaded 426,880 rows and 26 columns from ../data/vehicles.csv


### Candidate Features
A quick audit of the available columns and their properties helps identify viable predictors.


In [ ]:
candidate_features = [
    'price', 'year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel',
    'odometer', 'title_status', 'transmission', 'drive', 'size', 'type',
    'paint_color', 'state', 'region', 'lat', 'long', 'posting_date', 'description',
]
available = [col for col in candidate_features if col in df.columns]

feature_summary = pd.DataFrame({
    'dtype': df[available].dtypes.astype(str),
    'non_null': df[available].notna().sum(),
    'missing_pct': df[available].isna().mean().mul(100).round(2),
    'unique_values': df[available].nunique(dropna=True),
})
feature_summary = feature_summary.sort_values('missing_pct', ascending=False)
display(feature_summary)


### Feature Selection
To keep the eventual user experience focused, the analysis narrows in on **year**, **manufacturer**, **model**, and **fuel** as the primary inputs.


### Data Cleaning
Remove rows with missing values in the selected features, constrain prices to a reasonable range, and retain only the most frequently listed models to keep the training data balanced.


In [5]:
selected_features = ['year', 'manufacturer', 'model', 'fuel']

df = df.dropna(subset=[col for col in selected_features if col in df.columns])

price_min, price_max = 1000, 40000
df = df[(df['price'] >= price_min) & (df['price'] <= price_max)]

In [7]:
model_series = (
    df['model']
    .astype(str)
    .str.strip()
    .replace({'': pd.NA})
    .dropna()
)
top_models = model_series.value_counts().head(50)
top_model_set = set(top_models.index)

df = df[df['model'].astype(str).str.strip().isin(top_model_set)].copy()

## Pre-processing
Construct the feature matrix `X` and target vector `y` from `year`, `model`, `manufacturer`, and `fuel`, one-hot encoding the categorical fields to obtain a numeric design matrix.


In [8]:
target_column = 'price'
feature_columns = selected_features

y = pd.to_numeric(df[target_column], errors='coerce')
X_features = df[feature_columns].copy()
X_features['year'] = pd.to_numeric(X_features['year'], errors='coerce')

combined = pd.concat([X_features, y], axis=1).dropna()
y = combined[target_column]
X_features = combined[feature_columns]

categorical_cols = ['model', 'manufacturer', 'fuel']
X = pd.get_dummies(X_features, columns=categorical_cols, drop_first=True, dtype='int64')

print(f'Design matrix shape: {X.shape}')
print(f'Target vector length: {len(y)}')
display(X.head())
display(y.head())

Design matrix shape: (83777, 67)
Target vector length: 83777


,year,model_2500,model_accord,model_altima,model_camry,model_charger,model_cherokee,model_civic,model_corolla,model_corvette,model_cr-v,model_cruze,model_edge,model_elantra,model_equinox,model_escape,model_explorer,model_f-150,model_f-250,model_f150,model_focus,model_forester,model_fusion,model_grand caravan,model_grand cherokee,model_highlander,model_impala,model_jetta,model_malibu,model_mustang,model_odyssey,model_outback,model_pilot,model_prius,model_rav4,model_rogue,model_sentra,model_sienna,model_sierra,model_sierra 1500,model_silverado,model_silverado 1500,model_silverado 2500hd,model_sonata,model_soul,model_tacoma,model_tahoe,model_tundra,model_wrangler,model_wrangler unlimited,manufacturer_chrysler,manufacturer_dodge,manufacturer_ford,manufacturer_gmc,manufacturer_honda,manufacturer_hyundai,manufacturer_jeep,manufacturer_kia,manufacturer_nissan,manufacturer_ram,manufacturer_subaru,manufacturer_toyota,manufacturer_volkswagen,fuel_electric,fuel_gas,fuel_hybrid,fuel_other
28,2010.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
34,2019.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
37,1992.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
59,2012.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
159,2017.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


28     22590
34     35000
37      4500
59     14000
159    19000
Name: price, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=42
)

print(f'Training set: {X_train.shape[0]:,} rows, Test set: {X_test.shape[0]:,} rows')
display(X_train.head())
display(y_train.head())

Training set: 58,643 rows, Test set: 25,134 rows


,year,model_2500,model_accord,model_altima,model_camry,model_charger,model_cherokee,model_civic,model_corolla,model_corvette,model_cr-v,model_cruze,model_edge,model_elantra,model_equinox,model_escape,model_explorer,model_f-150,model_f-250,model_f150,model_focus,model_forester,model_fusion,model_grand caravan,model_grand cherokee,model_highlander,model_impala,model_jetta,model_malibu,model_mustang,model_odyssey,model_outback,model_pilot,model_prius,model_rav4,model_rogue,model_sentra,model_sienna,model_sierra,model_sierra 1500,model_silverado,model_silverado 1500,model_silverado 2500hd,model_sonata,model_soul,model_tacoma,model_tahoe,model_tundra,model_wrangler,model_wrangler unlimited,manufacturer_chrysler,manufacturer_dodge,manufacturer_ford,manufacturer_gmc,manufacturer_honda,manufacturer_hyundai,manufacturer_jeep,manufacturer_kia,manufacturer_nissan,manufacturer_ram,manufacturer_subaru,manufacturer_toyota,manufacturer_volkswagen,fuel_electric,fuel_gas,fuel_hybrid,fuel_other
100825,2001.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
94235,2002.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
399246,2015.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
304469,2018.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
279452,2011.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


100825     4795
94235      7500
399246    31900
304469    17298
279452    19995
Name: price, dtype: int64

## Modeling
Fit a baseline linear regression model on the processed data and report training and test RMSE and R² scores.


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

if 'X_train' not in globals():
    raise ValueError('Run the preprocessing and train/test split cells before modeling.')

model = LinearRegression()
model.fit(X_train, y_train)

def evaluate(features, target):
    predictions = model.predict(features)
    rmse = np.sqrt(mean_squared_error(target, predictions))
    r2 = r2_score(target, predictions)
    return rmse, r2

metrics = {
    'train': evaluate(X_train, y_train),
    'test': evaluate(X_test, y_test),
}

results_df = pd.DataFrame(
    {
        'RMSE': {k: v[0] for k, v in metrics.items()},
        'R^2': {k: v[1] for k, v in metrics.items()},
    }
)

display(results_df)

,RMSE,R^2
train,6369.476077,0.586427
test,6345.657529,0.589136


### AdaBoost Regressor
Evaluate a tree-based ensemble to capture non-linear interactions and benchmark against the linear baseline.


In [16]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

if 'X_train' not in globals():
    raise ValueError('Run the preprocessing and train/test split cells before modeling.')

ada_model = AdaBoostRegressor(random_state=42, n_estimators=600, learning_rate=0.05)
ada_model.fit(X_train, y_train)

def evaluate(features, target):
    predictions = ada_model.predict(features)
    rmse = np.sqrt(mean_squared_error(target, predictions))
    r2 = r2_score(target, predictions)
    return rmse, r2

metrics = {
    'train': evaluate(X_train, y_train),
    'test': evaluate(X_test, y_test),
}

ada_results_df = pd.DataFrame(
    {
        'RMSE': {k: v[0] for k, v in metrics.items()},
        'R^2': {k: v[1] for k, v in metrics.items()},
    }
)

display(ada_results_df)

,RMSE,R^2
train,7476.220393,0.430217
test,7455.409635,0.432863


### Ridge Regression
Apply L2 regularization to mitigate multicollinearity introduced by one-hot encoding.


In [12]:
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

alphas = np.logspace(-2, 2, num=20)
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train, y_train)

best_alpha = ridge_cv.alpha_
print(f'Best alpha from CV: {best_alpha:.4f}')

def evaluate(features, target):
    predictions = ridge_cv.predict(features)
    rmse = np.sqrt(mean_squared_error(target, predictions))
    r2 = r2_score(target, predictions)
    return rmse, r2

metrics = {
    'train': evaluate(X_train, y_train),
    'test': evaluate(X_test, y_test),
}

ridge_results_df = pd.DataFrame(
    {
        'RMSE': {k: v[0] for k, v in metrics.items()},
        'R^2': {k: v[1] for k, v in metrics.items()},
    }
)

display(ridge_results_df)

Best alpha from CV: 0.0100


,RMSE,R^2
train,6369.478906,0.586426
test,6345.649936,0.589137


### Random Forest Regressor
Assess a bagged tree ensemble that handles non-linear relationships with limited tuning effort.


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

if 'X_train' not in globals():
    raise ValueError('Run the preprocessing and train/test split cells before modeling.')

rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=4,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
)
rf_model.fit(X_train, y_train)

def evaluate(features, target):
    predictions = rf_model.predict(features)
    rmse = np.sqrt(mean_squared_error(target, predictions))
    r2 = r2_score(target, predictions)
    return rmse, r2

metrics = {
    'train': evaluate(X_train, y_train),
    'test': evaluate(X_test, y_test),
}

rf_results_df = pd.DataFrame(
    {
        'RMSE': {k: v[0] for k, v in metrics.items()},
        'R^2': {k: v[1] for k, v in metrics.items()},
    }
)

display(rf_results_df)

,RMSE,R^2
train,4144.326293,0.824913
test,4232.489659,0.817217


### Model Comparison
Consolidate train and test metrics from each model to compare performance side by side.

In [ ]:
model_frames = {
    'Linear Regression': 'results_df',
    'AdaBoost Regressor': 'ada_results_df',
    'Ridge Regression': 'ridge_results_df',
    'Random Forest Regressor': 'rf_results_df',
}

missing = [name for name in model_frames.values() if name not in globals()]
if missing:
    raise ValueError(
        'Run the individual model evaluation cells before building the comparison table.'
    )

comparison_rows = []
for model_name, frame_name in model_frames.items():
    df = globals()[frame_name].copy()
    df = df.rename_axis('Dataset').assign(Model=model_name).reset_index()
    comparison_rows.append(df)

model_comparison_df = (
    pd.concat(comparison_rows, ignore_index=True)
    .set_index(['Model', 'Dataset'])
    .sort_index()
)

display(model_comparison_df)

## Model Persistence
Persist the trained random forest model alongside preprocessing metadata (feature order, allowable categories, frequently listed models) so downstream applications can reproduce the transformations.


In [ ]:
import pickle
from pathlib import Path

if 'rf_model' not in globals():
    raise ValueError('Train the Random Forest model cell before persisting artifacts.')
if 'X' not in globals():
    raise ValueError('Run preprocessing cells to define feature matrix X.')

artifacts = {
    'model': rf_model,
    'feature_columns': X.columns.tolist(),
    'top_models': sorted(top_model_set) if 'top_model_set' in globals() else None,
    'categorical_columns': ['model', 'manufacturer', 'fuel'],
    'price_bounds': {'min': 1000, 'max': 40000},
}

models_dir = Path('models')
models_dir.mkdir(exist_ok=True)
artifact_path = models_dir / 'random_forest_regressor-v1.pkl'

with artifact_path.open('wb') as f:
    pickle.dump(artifacts, f)

print(f'Saved model and preprocessing metadata to {artifact_path.resolve()}')